In [1]:
import pandas as pd
import numpy as np
import logging
import time
import yaml
from psycopg2 import sql
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy.utils import complex_json_handler, unpack_data
from yfpy import get_logger

from scripts.db_psql_model import DatabaseCursor

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
TEAMS_FILE = list(Path().cwd().parent.glob("**/teams.yaml"))[0]

try:
    with open(PATH) as file:
        credentials = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

auth_dir = ""
league_id = 12514
game_id = 399
game_code = "nfl"
offline = False
all_output_as_json = False
consumer_key = credentials["YFPY_CONSUMER_KEY"]
consumer_secret = credentials["YFPY_CONSUMER_SECRET"]
browser_callback = True

_auth_dir = auth_dir
_consumer_key = str(consumer_key)
_consumer_secret = str(consumer_secret)
_browser_callback = browser_callback
league_id = str(league_id)
game_id = str(game_id)
game_code = str(game_code)
offline = offline
all_output_as_json = all_output_as_json

yahoo_query = YahooFantasySportsQuery(
    auth_dir=_auth_dir,
    league_id=league_id,
    game_id=game_id,
    game_code=game_code,
    offline=offline,
    all_output_as_json=all_output_as_json,
    consumer_key=_consumer_key,
    consumer_secret=_consumer_secret,
    browser_callback=_browser_callback,
)

f_matchups = pd.DataFrame()

for nfl_week in range(1, 17):
    m = []
    team_a = pd.DataFrame()
    team_b = pd.DataFrame()

    response = yahoo_query.get_league_matchups_by_week(nfl_week)

    for data in response:
        m.append(complex_json_handler(data["matchup"]))

    matchups = pd.DataFrame()
    for r in m:
        matchup = pd.json_normalize(r)
        matchup = matchup[
            [
                "is_consolation",
                "is_matchup_recap_available",
                "is_playoffs",
                "is_tied",
                "matchup_recap_title",
                "matchup_recap_url",
                "status",
                "week",
                "week_end",
                "week_start",
                "winner_team_key",
            ]
        ]
        try:
            team_a = pd.json_normalize(
                complex_json_handler(r["matchup_grades"][0]["matchup_grade"])
            )
            team_a["points"] = complex_json_handler(r["teams"][0]["team"])[
                "team_points"
            ]["total"]
            team_a["projected_points"] = complex_json_handler(r["teams"][0]["team"])[
                "team_projected_points"
            ]["total"]

        except:
            team_a = pd.json_normalize(complex_json_handler(r["teams"][0]["team"]))
            team_a = team_a[
                ["team_key", "team_points.total", "team_projected_points.total"]
            ]
            team_a["grade"] = ""
            team_a.rename(
                columns={
                    "team_points.total": "points",
                    "team_projected_points.total": "projected_points",
                },
                inplace=True,
            )

        team_a = team_a.add_prefix("team_a_")

        try:
            team_b = pd.json_normalize(
                complex_json_handler(r["matchup_grades"][1]["matchup_grade"])
            )
            team_b["points"] = complex_json_handler(r["teams"][1]["team"])[
                "team_points"
            ]["total"]
            team_b["projected_points"] = complex_json_handler(r["teams"][1]["team"])[
                "team_projected_points"
            ]["total"]

        except:
            team_b = pd.json_normalize(complex_json_handler(r["teams"][1]["team"]))
            team_b = team_b[
                ["team_key", "team_points.total", "team_projected_points.total"]
            ]
            team_b["grade"] = ""
            team_b.rename(
                columns={
                    "team_points.total": "points",
                    "team_projected_points.total": "projected_points",
                },
                inplace=True,
            )

        team_b = team_b.add_prefix("team_b_")

        matchup = pd.concat([matchup, team_a, team_b], axis=1)

        matchups = pd.concat([matchups, matchup])

    try:
        matchups.drop(["teams", "matchup_grades"], axis=1, inplace=True)

    except:
        pass

    matchups["game_id"] = game_id
    matchups["league_id"] = league_id
    matchups["is_playoffs"].fillna(0, inplace=True)
    matchups["is_consolation"].fillna(0, inplace=True)
    matchups["is_tied"].fillna(0, inplace=True)

    matchups = matchups[
        [
            "game_id",
            "league_id",
            "week",
            "week_start",
            "week_end",
            "is_playoffs",
            "is_consolation",
            "is_tied",
            "team_a_team_key",
            "team_a_points",
            "team_a_projected_points",
            "team_b_team_key",
            "team_b_points",
            "team_b_projected_points",
            "winner_team_key",
            "team_a_grade",
            "team_b_grade",
        ]
    ]

    matchups["game_id"] = matchups["game_id"].astype(int)
    matchups["league_id"] = matchups["league_id"].astype(int)
    matchups["week"] = matchups["week"].astype(int)
    matchups["week_start"] = matchups["week_start"].astype("datetime64[D]")
    matchups["week_end"] = matchups["week_end"].astype("datetime64[D]")
    matchups["is_playoffs"] = matchups["is_playoffs"].astype("bool")
    matchups["is_consolation"] = matchups["is_consolation"].astype("bool")
    matchups["is_tied"] = matchups["is_tied"].astype("bool")
    matchups["team_a_team_key"] = matchups["team_a_team_key"].astype(str)
    matchups["team_a_points"] = (
        matchups["team_a_points"].astype(float).round(decimals=2)
    )
    matchups["team_a_projected_points"] = (
        matchups["team_a_projected_points"].astype(float).round(decimals=2)
    )
    matchups["team_b_team_key"] = matchups["team_b_team_key"].astype(str)
    matchups["team_b_points"] = (
        matchups["team_b_points"].astype(float).round(decimals=2)
    )
    matchups["team_b_projected_points"] = (
        matchups["team_b_projected_points"].astype(float).round(decimals=2)
    )
    matchups["winner_team_key"] = matchups["winner_team_key"].astype(str)
    matchups["team_a_grade"] = matchups["team_a_grade"].astype(str)
    matchups["team_b_grade"] = matchups["team_b_grade"].astype(str)
    f_matchups = pd.concat([f_matchups, matchups])

In [2]:
f_matchups

,game_id,league_id,week,week_start,week_end,is_playoffs,is_consolation,is_tied,team_a_team_key,team_a_points,team_a_projected_points,team_b_team_key,team_b_points,team_b_projected_points,winner_team_key,team_a_grade,team_b_grade
0,399,12514,1,2020-09-10,2020-09-14,False,False,False,399.l.12514.t.1,110.10,97.79,399.l.12514.t.2,101.60,123.12,399.l.12514.t.1,B+,B
0,399,12514,1,2020-09-10,2020-09-14,False,False,False,399.l.12514.t.3,124.38,122.05,399.l.12514.t.9,132.64,125.51,399.l.12514.t.9,A,A
0,399,12514,1,2020-09-10,2020-09-14,False,False,False,399.l.12514.t.4,126.70,116.75,399.l.12514.t.8,108.30,124.89,399.l.12514.t.4,A+,B
0,399,12514,1,2020-09-10,2020-09-14,False,False,False,399.l.12514.t.5,117.48,116.11,399.l.12514.t.7,111.54,126.07,399.l.12514.t.5,A,A-
0,399,12514,1,2020-09-10,2020-09-14,False,False,False,399.l.12514.t.6,130.70,121.39,399.l.12514.t.10,100.78,119.82,399.l.12514.t.6,A+,C+
0,399,12514,2,2020-09-15,2020-09-21,False,False,False,399.l.12514.t.1,125.18,115.44,399.l.12514.t.3,105.26,118.96,399.l.12514.t.1,B,C
0,399,12514,2,2020-09-15,2020-09-21,False,False,False,399.l.12514.t.2,112.32,116.59,399.l.12514.t.10,113.22,112.28,399.l.12514.t.10,B,B-
0,399,12514,2,2020-09-15,2020-09-21,False,False,False,399.l.12514.t.4,117.38,119.66,399.l.12514.t.9,109.70,126.84,399.l.12514.t.4,B,C
0,399,12514,2,2020-09-15,2020-09-21,False,False,False,399.l.12514.t.5,121.80,117.01,399.l.12514.t.8,119.76,125.66,399.l.12514.t.5,B,B-
0,399,12514,2,2020-09-15,2020-09-21,False,False,False,399.l.12514.t.6,197.44,127.52,399.l.12514.t.7,146.18,120.34,399.l.12514.t.6,A+,A


In [12]:
import yaml
import numpy as np
import pandas as pd
from pathlib import Path
from scripts.db_psql_model import DatabaseCursor

date = np.datetime64("2023-01-07", "D")
PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
nfl_weeks_query = "SELECT * FROM dev.nfl_weeks"
nfl_weeks = DatabaseCursor(PATH, option_schema="raw").copy_data_from_postgres(
    nfl_weeks_query
)
nfl_weeks["start"] = nfl_weeks["start"].astype("datetime64[D]")
nfl_weeks["end"] = nfl_weeks["end"].astype("datetime64[D]")
game_ids = nfl_weeks["game_id"][(nfl_weeks["end"].astype("datetime64[D]") > date)]
game_id = game_ids.min()
week_1 = nfl_weeks[(nfl_weeks["game_id"] == game_id) & (nfl_weeks["week"] == 1)]
season = week_1["start"].values[0].astype("datetime64[Y]").astype(int) + 1970


----Successfully pulled: SELECT * FROM dev.nfl_weeks



In [1]:
from scripts.utils import log_print

error = """SAMPLE ERROR"""
module_ = "test.py"
class_ = "TestClass"
method_ = "test_method"
option1 = "option1"
option2 = "option2"
option3 = "option3"
log_print(
    error=error,
    module=module_,
    class_=class_,
    method=method_,
    option1=option1,
    option2=option2,
    option3=option3,
)

In [7]:
import numpy as np

test = 1
if not np.isnan(test):
    print(True, test)

True 1
